In [6]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
Example script to create a choropleth map of US states
showing air pollution deaths per 10,000 residents (fictional data),
mimicking the style from a provided reference image.
"""

import os
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

# 1. FICTIONAL DATA for demonstration
data = [
    {'state': 'Alabama',       'abbr': 'AL', 'air_deaths_per_10k': 2.3},
    {'state': 'Alaska',        'abbr': 'AK', 'air_deaths_per_10k': 0.9},
    {'state': 'Arizona',       'abbr': 'AZ', 'air_deaths_per_10k': 1.4},
    {'state': 'Arkansas',      'abbr': 'AR', 'air_deaths_per_10k': 1.9},
    {'state': 'California',    'abbr': 'CA', 'air_deaths_per_10k': 3.1},
    {'state': 'Colorado',      'abbr': 'CO', 'air_deaths_per_10k': 1.1},
    {'state': 'Connecticut',   'abbr': 'CT', 'air_deaths_per_10k': 1.8},
    {'state': 'Delaware',      'abbr': 'DE', 'air_deaths_per_10k': 2.2},
    {'state': 'Florida',       'abbr': 'FL', 'air_deaths_per_10k': 2.0},
    {'state': 'Georgia',       'abbr': 'GA', 'air_deaths_per_10k': 2.1},
    {'state': 'Hawaii',        'abbr': 'HI', 'air_deaths_per_10k': 1.2},
    {'state': 'Idaho',         'abbr': 'ID', 'air_deaths_per_10k': 1.0},
    {'state': 'Illinois',      'abbr': 'IL', 'air_deaths_per_10k': 2.7},
    {'state': 'Indiana',       'abbr': 'IN', 'air_deaths_per_10k': 2.4},
    {'state': 'Iowa',          'abbr': 'IA', 'air_deaths_per_10k': 1.5},
    {'state': 'Kansas',        'abbr': 'KS', 'air_deaths_per_10k': 1.3},
    {'state': 'Kentucky',      'abbr': 'KY', 'air_deaths_per_10k': 2.6},
    {'state': 'Louisiana',     'abbr': 'LA', 'air_deaths_per_10k': 2.5},
    {'state': 'Maine',         'abbr': 'ME', 'air_deaths_per_10k': 1.3},
    {'state': 'Maryland',      'abbr': 'MD', 'air_deaths_per_10k': 1.9},
    {'state': 'Massachusetts', 'abbr': 'MA', 'air_deaths_per_10k': 1.6},
    {'state': 'Michigan',      'abbr': 'MI', 'air_deaths_per_10k': 2.8},
    {'state': 'Minnesota',     'abbr': 'MN', 'air_deaths_per_10k': 2.2},
    {'state': 'Mississippi',   'abbr': 'MS', 'air_deaths_per_10k': 2.1},
    {'state': 'Missouri',      'abbr': 'MO', 'air_deaths_per_10k': 2.7},
    {'state': 'Montana',       'abbr': 'MT', 'air_deaths_per_10k': 1.1},
    {'state': 'Nebraska',      'abbr': 'NE', 'air_deaths_per_10k': 1.3},
    {'state': 'Nevada',        'abbr': 'NV', 'air_deaths_per_10k': 1.5},
    {'state': 'New Hampshire', 'abbr': 'NH', 'air_deaths_per_10k': 1.4},
    {'state': 'New Jersey',    'abbr': 'NJ', 'air_deaths_per_10k': 2.3},
    {'state': 'New Mexico',    'abbr': 'NM', 'air_deaths_per_10k': 1.0},
    {'state': 'New York',      'abbr': 'NY', 'air_deaths_per_10k': 3.2},
    {'state': 'North Carolina','abbr': 'NC', 'air_deaths_per_10k': 2.5},
    {'state': 'North Dakota',  'abbr': 'ND', 'air_deaths_per_10k': 1.2},
    {'state': 'Ohio',          'abbr': 'OH', 'air_deaths_per_10k': 2.9},
    {'state': 'Oklahoma',      'abbr': 'OK', 'air_deaths_per_10k': 1.6},
    {'state': 'Oregon',        'abbr': 'OR', 'air_deaths_per_10k': 1.8},
    {'state': 'Pennsylvania',  'abbr': 'PA', 'air_deaths_per_10k': 3.0},
    {'state': 'Rhode Island',  'abbr': 'RI', 'air_deaths_per_10k': 1.7},
    {'state': 'South Carolina','abbr': 'SC', 'air_deaths_per_10k': 2.2},
    {'state': 'South Dakota',  'abbr': 'SD', 'air_deaths_per_10k': 1.4},
    {'state': 'Tennessee',     'abbr': 'TN', 'air_deaths_per_10k': 2.6},
    {'state': 'Texas',         'abbr': 'TX', 'air_deaths_per_10k': 2.0},
    {'state': 'Utah',          'abbr': 'UT', 'air_deaths_per_10k': 1.0},
    {'state': 'Vermont',       'abbr': 'VT', 'air_deaths_per_10k': 1.0},
    {'state': 'Virginia',      'abbr': 'VA', 'air_deaths_per_10k': 2.1},
    {'state': 'Washington',    'abbr': 'WA', 'air_deaths_per_10k': 1.9},
    {'state': 'West Virginia', 'abbr': 'WV', 'air_deaths_per_10k': 2.4},
    {'state': 'Wisconsin',     'abbr': 'WI', 'air_deaths_per_10k': 2.5},
    {'state': 'Wyoming',       'abbr': 'WY', 'air_deaths_per_10k': 1.1},
]

df_data = pd.DataFrame(data)

# 2. PATH TO A REAL SHAPEFILE
shapefile_path = '/Users/jorgejimenez/Documents/UP/DataScienceB/CLASE1Y2/hw/AirPollutionDeathsGraphLN-superJumbo.webp'  # <== CHANGE THIS!

if not os.path.exists(shapefile_path):
    raise FileNotFoundError(f"Shapefile not found at {shapefile_path}")

# 3. READ SHAPEFILE WITH GEOPANDAS
states_gdf = gpd.read_file(shapefile_path)

# Filter down to only the states we have data for
states_gdf = states_gdf[states_gdf['STUSPS'].isin(df_data['abbr'].tolist())].copy()

# 4. MERGE THE SHAPEFILE WITH OUR DATA
merged_gdf = states_gdf.merge(
    df_data,
    left_on='STUSPS',    # 'STUSPS' is typically the state postal code column
    right_on='abbr',
    how='left'
)

# 5. PLOT THE CHOROPLETH
fig, ax = plt.subplots(figsize=(12, 8))
merged_gdf.plot(
    column='air_deaths_per_10k',
    cmap='Purples',
    linewidth=0.8,
    edgecolor='white',
    legend=True,
    legend_kwds={'label': "Air pollution deaths per 10,000 residents"},
    ax=ax
)

# Optional styling
ax.set_axis_off()
ax.set_title(
    "U.S. Air Pollution Deaths (Fictional Data)",
    fontdict={'fontsize': 16},
    pad=20
)

plt.show()


AttributeError: module 'fiona' has no attribute 'path'